In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from Bio import Seq, SeqIO, Align, pairwise2

In [ ]:
def import_pair_reads(directory, fastq1_name, fastq2_name):
    """
    Importing the fastq sequences and their reverse complement
    """
    
    reads1, reads2 = [], []
    q1, q2 = [], []
    
    fastq1 = SeqIO.parse(directory+fastq1_name, "fastq")
    for record in fastq1:
        reads1.append(record.seq)
        q1.append(record.letter_annotations["phred_quality"])

    fastq2 = SeqIO.parse(directory+fastq2_name, "fastq")
    for record in fastq2:
        reads2.append(record.seq)
        q2.append(record.letter_annotations["phred_quality"])
    
    return reads1, reads2, q1, q2


def join_by_q(r1, r2, q1, q2):
    """
    Given two sequences and the qualities at each position it returns the sequence
    having the letter with larger quality and the best qualities
    """
    q1, q2 = np.array(q1), np.array(q2)
    res =  np.where(q1>q2, list(r1), list(r2))
    qs = np.where(q1>q2, q1, q2)
    return ''.join(res), qs


def find_first_nucl(seq):
    iA, iC, iT, iG = seq.find('A'), seq.find('C'), seq.find('T'), seq.find('G')
    return min(iA, iC, iT, iG)

In [ ]:
def join_pairs(align, q1, q2):
    
    start_i = find_first_nucl(align.seqB)
    # Case in which there is no gap in the reversed complement at the beginning
    # This means that 
    if start_i==0:
        return [], []
    
    # Add the first non-overlapping seqA
    start_i_qa, start_i_qb = start_i, 0
    seq, qs = align.seqA[:start_i], np.array(q1[:start_i])
    
    # Iterating over the indexes in the overlapping part
    # start_i - last_i is the largest window of indexes without gaps
    while start_i < len(align.seqA)-1:

        # computing the positions of the overlap chunk without gaps
        gap_indA, gap_indB = align.seqA[start_i:].find('-'), align.seqB[start_i:].find('-')
        if gap_indA == -1: gap_indA = len(align.seqA)
        else: gap_indA += start_i
        if gap_indB == -1: gap_indB = len(align.seqB)
        else: gap_indB += start_i
        last_i = min(gap_indA, gap_indB)
        delta = last_i-start_i
        
        # Joining the overlap chunk before the first gap
        sA, sB = align.seqA[start_i:last_i], align.seqB[start_i:last_i]
        qA, qB = q1[start_i_qa:start_i_qa+delta], q2[start_i_qb:start_i_qb+delta]
        new_s, new_qs = join_by_q(sA, sB, qA, qB)
        seq += new_s
        qs = np.append(qs, new_qs)
        start_i = last_i+1

        # Resolving the gap found
        if gap_indA < gap_indB:
            seq += align.seqB[start_i-1]
            start_i_qa += delta
            start_i_qb += delta+1
            qs = np.append(qs, q2[start_i_qb-1])
        elif gap_indA > gap_indB:
            seq += align.seqA[start_i-1]
            start_i_qa += delta+1
            start_i_qb += delta
            qs = np.append(qs, q1[start_i_qa-1])
        elif gap_indA == len(align.seqA):
            break
        else:
            seq += align.seqA[start_i-1]
            print('Double gap found')

        # Final gap of the sequence B
        if set(align.seqA[start_i:]) == {'-'}:
            seq += align.seqB[start_i:]
            qs = np.append(qs, q2[start_i_qb:])
            #print(start_i, len(align.seqB))
            break
            
    return seq, qs


def filter_seqs(seqs, av_qs):
    # Discarding for quality overlap and length window
    to_keep_q, to_keep_l = set(), set()
    for i in range(len(av_qs)):
        if av_qs[i] > q_threshold:
            to_keep_q.add(i)
        #if align_scores[i] > s_threshold:
        #    to_keep_s.add(i)
        if len(seqs[i]) > len_bounds[0] and len(seqs[i]) < len_bounds[1]:
            to_keep_l.add(i)
    filt_seqs = np.take(seqs, list(to_keep_q.intersection(to_keep_l)))

    # Discarding singletons
    uniq_seqs, index, counts = np.unique(filt_seqs, return_counts=True, return_inverse=True)
    to_keep_sing = set()
    for i in range(len(index)):
        if counts[index[i]] > 1:
            to_keep_sing.add(i)
    final_seqs = np.take(filt_seqs, list(to_keep_sing))
    
    return final_seqs, len(seqs)-len(to_keep_l), len(seqs)-len(to_keep_q), len(filt_seqs)-len(to_keep_sing)


def build_seq(reads1, reads2, quals1, quals2, gap_penalty=-20):
    """
    It assemble the reads1 with their reverse complement reads2 and the sequencing 
    quality at each poistion.
    """
    seqs = []
    av_qs = []
    align_errors = []
    #index_map = []

    for i in range(len(reads1)):
        read1, read2 = reads1[i], Seq.reverse_complement(reads2[i])
        qual1, qual2 = quals1[i], quals2[i][::-1]
        if set(read1) != {'A','C','G','T'} or set(read2) != {'A','C','G','T'}:
            align_errors.append(i)
            continue
            
        alignment = pairwise2.align.localms(read1, read2, 1, -1, gap_penalty, gap_penalty)[0]
        new_seq, new_qs = join_pairs(alignment, qual1, qual2)
        if new_seq == []:
            align_errors.append(i)
        else:
            seqs.append(new_seq)
            av_qs.append(new_qs.mean())
            #index_map.append(i)
            #align_scores.append(alignment.score/(alignment.end-alignment.start))
            
    return (*filter_seqs(seqs, av_qs), len(align_errors))

In [ ]:
def export_seqs(uniq_seqs, counts, name, directory):
    f = open(directory+name+'.fasta', 'w')
    for i in range(len(counts)):
        f.write('>'+str(i+1)+'_'+str(counts[i])+'\n')
        f.write(uniq_seqs[i]+'\n')
    f.close()

In [ ]:
gap_penalty = -20
q_threshold = 30
len_bounds = [400, 425]

In [ ]:
info = pd.DataFrame(columns=['id', 'N_seqs', 'N_discarderd_singl', 'N_discarderd_len', 'N_discarded_quality', \
                             'N_discarded_align', 'N_discarded_overl', 'av_gene_len', \
                             'std_gene_len'])

In [ ]:
for fastq in os.listdir('../fastq/hiv/'):
    
    if fastq.split('_')[-1] == '1.fastq':
        id_ = fastq.split('_')[0]+'_'+fastq.split('_')[1]
        print(id_)
        
        if id_+'_hiv.fasta' in os.listdir('../hiv_seqs/'):
            print('already present, skip.')
            continue
        
        name1, name2 = id_+'_hiv_1.fastq', id_+'_hiv_2.fastq'
        reads1, reads2, quals1, quals2 = import_pair_reads('../fastq/hiv/', name1, name2)
        seqs, l_fail, q_fail, sing_fail, a_fail = build_seq(reads1, reads2, quals1, quals2)
        
        info_d = {'id' : id_}
        uniq_seqs, counts = np.unique(seqs, return_counts=True)
        
        info_d['N_seqs']= len(seqs)
        info_d['N_unique_seqs']= len(uniq_seqs)
        info_d['N_discarderd_len'] = l_fail
        info_d['N_discarderd_singl'] = sing_fail
        #info_d['N_discarded_overl'] = overl_fail
        info_d['N_discarded_align'] = a_fail
        info_d['N_discarded_quality'] = q_fail
        lens = [len(s) for s in seqs]
        info_d['av_gene_len'] = np.mean(lens)
        info_d['std_gene_len'] = np.std(lens)
        info = info.append(info_d, ignore_index=True)
        
        if len(uniq_seqs) > 0:
            export_seqs(uniq_seqs, counts, id_+'_hiv', '../hiv_seqs/')

In [ ]:
info.sort_index().to_csv('../hiv_seqs_info.tsv',sep='\t')